In [1]:
from qiskit import *
from qiskit.visualization import *
from math import pi
import numpy as np
from random import random

## For 1-qubit Hamiltonian

                                        H = a.I + b.X + c.Y + d.Z

* **Hamiltonian Mapping:** -
* **Initial State:** Zero
* **Ansatz (PQC):** RxRy
* **Optimizer:** Powell
* **Noise:** -

In [2]:
# Ansatz must represent different states in Hilbert Space.
# These Ry and Rx gates allow representation of any state on a bloch sphere. 
# Rotations are performed around both the x-axis and the y-axis.

# We could choose a different ansatz here as all the states on a bloch sphere are not needed to be represented for generating a trial wavefinction in this case.

def ansatz(circuit, parameter):  
    circuit.ry(parameter[1], 0)    # A rotation around y-axis
    circuit.rx(parameter[0], 0)    # A rotation around x-axis
    
    return circuit

In [3]:
def change_basis(circuit, hamiltonian_term):
    if hamiltonian_term == 'X':
        circuit.h(0)
        # or use circuit.u2(0, pi, 0)
    if hamiltonian_term == 'Y':
        circuit.u2(0, pi/2, 0)
        
    return circuit

In [4]:
def measure_expectation(n, circuit, hamiltonian_term):
    if hamiltonian_term == 'I':
        return 1
    if hamiltonian_term == 'Z':
        circuit.measure(range(n), range(n))
    if hamiltonian_term == 'X':
        change_basis(circuit, hamiltonian_term)
        circuit.measure(range(n), range(n))
    if hamiltonian_term == 'Y':
        change_basis(circuit, hamiltonian_term)
        circuit.measure(range(n), range(n))
        
    backend = Aer.get_backend('qasm_simulator')
    results = execute(circuit, backend, shots = 1024).result()
    counts = results.get_counts()
    # print(counts)
    # plot_histogram(counts)
        
    expectation_val = 0

    for i in counts:
        sign = 1
        if i == '1':
            sign = -1
            expectation_val += (sign * counts[i])

    expectation_val = expectation_val/1024
    expectation_val = expectation_val
    # print(expectation_val)
    
    return expectation_val

In [5]:
def classical_adder(n, circuit, coefficients):
    quantum_module_I = coefficients['I'] * measure_expectation(n, circuit, 'I')
    quantum_module_Z = coefficients['Z'] * measure_expectation(n, circuit, 'Z')
    quantum_module_Y = coefficients['Y'] * measure_expectation(n, circuit, 'Y')
    quantum_module_X = coefficients['X'] * measure_expectation(n, circuit, 'X')

    classical_adder_result = quantum_module_I + quantum_module_X + quantum_module_Y + quantum_module_Z
    
    return classical_adder_result

In [6]:
def main(parameter, n, Hamiltonian, coefficients):
    circuit = QuantumCircuit(n, n)
    
    # Ansatz
    ansatz(circuit, parameter)
    
    circuit.barrier()
    
    # Apply Change basis + Measurement + Expectation Value
    classical_adder_result = classical_adder(n, circuit, coefficients)
        
    return classical_adder_result

In [14]:
from qiskit.aqua.operators import WeightedPauliOperator

scale = 10
# coefficients = {'I': 0.7, 'X': 0.6, 'Y': 0.5, 'Z': 0.1}
coefficients = {'I': scale * random(), 'X': scale * random(), 'Y': scale * random(), 'Z': scale * random()}
print(coefficients)

{'I': 7.949215609719808, 'X': 6.799722129678721, 'Y': 2.609278996035881, 'Z': 7.00401591904898}


In [15]:
n = 1
parameter = np.array([np.pi, np.pi])
# parameter = np.array([np.pi]) or just np.array([pi])
# Why need a numpy array here?

Hamiltonian = ['I', 'X', 'Y', 'Z']

vqe_result = main(parameter, n, Hamiltonian, coefficients)
print(vqe_result)

3.440373513694437


<ipython-input-3-5cbb27ac4c56>:6: DeprecationWarning: The QuantumCircuit.u2 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You can use the general 1-qubit gate QuantumCircuit.u instead: u2(φ,λ) = u(π/2, φ, λ). Alternatively, you can decompose it interms of QuantumCircuit.p and QuantumCircuit.sx: u2(φ,λ) = p(π/2+φ) sx p(λ-π/2) (1 pulse on hardware).
  circuit.u2(0, pi/2, 0)


### Minimization / Optimization

In [ ]:
# !!!!!!!! IMPORTANT !!!!!!!
# For this minimize function to work, the func to be minimized needs to have this form:
# func(array, *args)

# minimize(function (to be minimized), array (initial guess), args = (additional arguments of the function), method)
# Refer this: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html

In [16]:
from scipy.optimize import minimize

tol = 1e-3    # Tolerance
result = minimize(main, parameter, args = (n, Hamiltonian, coefficients), method = 'Powell')
# print(result)
print(result.fun)

-3.803259439993395


**Refer:** https://www.mustythoughts.com/variational-quantum-eigensolver-explained

**Refer:** https://github.com/DavitKhach/quantum-algorithms-tutorials/blob/master/variational_quantum_eigensolver.ipynb

For scipy.optimize.minimize function, **Refer:** https://github.com/scipy/scipy/blob/master/scipy/optimize/_minimize.py